## Estimation of pi

It is know by almost all quantum computing students the phase estimation algorithm. Here we'll present an application of this algorithm by estimating the value of pi

First we have to import a few necessary packages.

In [10]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.circuit.library import QFT
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math
import numpy as np
from iqx import *
import operator

Here we define an implementation of the QPE estimation. If you want to learn more about this algorithm check out this post.

In [11]:
class QPE():
    def __init__(self, n):
        self.n = n
        self.qc = QuantumCircuit(n+1, n)
        for qubit in range(n):
            self.qc.h(qubit)
        self.qc.x(n)
    
    def calculate_phase(self):
        for x in range(self.n):
            exponent = 2**(self.n-x-1)
            self.unitary_operator_exponent(self.qc, x, self.n, exponent)
        self.qc.append(QFT( self.n, do_swaps=True).inverse(), range(self.n))
        self.qc.measure(range(self.n), range(self.n))
        simulator = Aer.get_backend('qasm_simulator')
        counts = execute(self.qc, backend=simulator, shots=10000).result().get_counts(self.qc)
        plot_histogram(counts)
        highest_probability_outcome = max(counts.items(), key=operator.itemgetter(1))[0][::-1]
        measured_theta = int(highest_probability_outcome, 2)/2**self.n
        return measured_theta
    
    def unitary_operator_exponent(self, given_circuit, control_qubit, target_qubit,exponent):
        for e in range(exponent):   
            self.unitary_operator(given_circuit, control_qubit, target_qubit)
            
    def unitary_operator(self,given_circuit, control_qubit, target_qubit):
        given_circuit.cp(1, target_qubit, control_qubit)    
    


Then we apply the function to estimate the phase. When we obtain the phase we must apply an identity to obtain the value of pi.

In [12]:
qpe = QPE(12)
measured_theta = qpe.calculate_phase()
pi = 1/(2*(measured_theta))
print("pi value is: ", pi)

pi value is:  3.1411042944785277


As many other quantum algorithms is not a perfect method to estimate pi because simulating a quantum computer is computational costly (exponential in the number of qubits). And if you run this algorithm on an actual quantum computer the noise will affect the calculations so won't have a good result. So for now this application is just a scientific curiosity of the algorithm.